In [39]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./funlopt/funl_dynamics.jl")
include("./funlopt/funl_ctcs.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_constraint.jl")

impose! (generic function with 3 methods)

In [40]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/unicycle_ICV_0408" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"]
Qnom = my_dict["Q"]
Ynom = my_dict["Y"]
Znom = my_dict["Z"]
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end

In [41]:
ix = 3
iu = 2
iϕ = 2
iv = 2
iψ = iϕ*iv
iμ = iψ
@assert size(xnom,2) - 1 == N
iq = ix^2

9

In [42]:
dynamics = Unicycle()
ix = dynamics.ix
iu = dynamics.iu
alpha = 0.0
dynamics.β .= [2.0,2.0]
θ0 = 0.5
# DLMI = LinearSOH(alpha,ix,iu)
DLMI = LinearDLMI(alpha,ix,iu)

LinearDLMI(0.0, 3, 2, 9, 6, 0, [1 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 1], [1 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 1])

In [43]:
ctcs = Invariance(1.0,1e-4,1.2,θ0,dynamics,DLMI)
# ctcs = QPD(1.0,1e-4,1.2,dynamics,DLMI)
# ctcs = BoundB(10.0,1e-4,1.2,θ0,dynamics,DLMI)
idx = 4
# Qtest = Qnom[:,:,idx]
Qtest = 1*[1 0.1 0.2;0.1 2 0.3;0.2 0.3 -0.2]
A,B = diff(dynamics,xnom[:,idx],unom[:,idx])

([0.0 0.0 -0.8563473835516268; 0.0 0.0 1.2805836524424632; 0.0 0.0 0.0], [0.831263029672674 0.0; 0.5558792814086592 0.0; 0.0 1.0])

In [44]:
Aq,Sy,Sz,Sb = diff(ctcs,vec(Qtest),vec(Ynom[:,:,idx]),vec(Znom[:,:,idx]),1.0,A,B)
Fq,Fy,Fz,Fb = diff_numeric(ctcs,vec(Qtest),vec(Ynom[:,:,idx]),vec(Znom[:,:,idx]),1.0,A,B)

([0.0 0.0 … 0.0 -0.25534866235354947], [-0.24434609105505878 0.0 … 0.1352759815276272 0.0], [-0.178123348987258 -0.20781004386738466 … 0.11504873126400028 -0.054594821247144765], [0.17044900274543195;;])

In [45]:
Aq

1×9 Matrix{Float64}:
 0.0  0.0  0.461231  0.0  0.0  0.538101  0.0  0.0  -0.255349

In [46]:
Fq

1×9 Matrix{Float64}:
 0.0  0.0  0.461231  0.0  0.0  0.538101  0.0  0.0  -0.255349

In [47]:
sum(abs.(Aq - Fq))

5.3260285071132785e-11

In [48]:
sum(abs.(Sy - Fy))

2.4363441819552634e-10

In [49]:
sum(abs.(Sz - Fz))

1.3992682776353682e-9

In [50]:
sum(abs.(Sb - Fb))

2.7507718325381347e-11

In [52]:
ctcs = QPD(1,1e3,1e-6,ix,iu)
dynamics = Unicycle()
alpha = 0.7
funl_dynamics = LinearDLMI(alpha,ix,iu)

AssertionError: AssertionError: typeof(funl_dynamics) == LinearDLMI

In [53]:
(
    Aq,Bm,Bp,Sm,Sp
) = discretize_foh(funl_dynamics,
    dynamics,xnom[:,1:N],unom,dtnom,Qnom[:,:,1:N],Ynom,Znom)

UndefVarError: UndefVarError: `funl_dynamics` not defined

In [54]:
include("./funlopt/funl_ctcs.jl")

In [55]:
(
    Aq_,Bym,Byp,Bzm,Bzp,Sq,Sym,Syp,Szm,Szp,SZ,x_prop,q_prop,s_prop
) = discretize_foh(funl_dynamics,dynamics,ctcs,
xnom[:,1:N],unom,dtnom,Qnom[:,:,1:N],Ynom,Znom)

UndefVarError: UndefVarError: `funl_dynamics` not defined

In [56]:
function get_error(A,B)
    return sum(abs.(A-B))
end
println(get_error(Aq,Aq_))
println(get_error(Bm,Bym))
println(get_error(Bp,Byp))
println(get_error(Sm,Bzm))
println(get_error(Sp,Bzp))
println(get_error(Sp,Bzp))

UndefVarError: UndefVarError: `Aq_` not defined

In [57]:
SZ

UndefVarError: UndefVarError: `SZ` not defined